# 04 Inference and Verification on the MCU

Once the neural network has been deployed on the MCU, it waits for inference data via the serial interface. This script enables the connection using the serial interface, provides the input data, and saves the inference result returned by the MCU.

Finally, the last step enables a complete verification of the testset by providing the input data one by one and saving the inference results. This allows to verify the accuracy on the MCU and, therefore, check for inconsistencies.

In [1]:
%run '00_README.ipynb'
%run 'H02_TFL-Conversion.ipynb'
%run 'H04_MCU-Verification.ipynb'

Imported helper functions from 00_README.ipynb
Imported all modules.
	Tensorflow Version:  2.2.0
	Numpy Version:  1.19.0
	Pandas Version:  1.0.5
Imported helper functions from H02_TFL-Conversion.ipynb
Imported helper functions from H04_MCU-Verification.ipynb


In [2]:
display(model_selection)

Dropdown(description='Select model:', options=('keras-model/01a_LeNet-MNIST.h5', 'keras-model/01a_LeNet-MNIST_…

In [3]:
# load model
tf_model_file = model_selection.value
tf_model = tf.keras.models.load_model(tf_model_file)

model_name = get_tf_model_string(tf_model_file)

In [4]:
 data_selection = widgets.Dropdown(
    options=sorted(glob.glob("keras-model/*.py")),
    description='Select model:',
    layout=Layout(width='100%')
)
display(data_selection)

Dropdown(description='Select model:', layout=Layout(width='100%'), options=('keras-model/01a_LeNet-MNIST_data.…

In [5]:
tf_model_data = data_selection.value
%run -i {tf_model_data}

The input length is 4096 and the output length 10


## Set port

In [61]:
port = set_port('/dev/tty.usbmodem145103', 1000000)

## Run a single test inference

In [63]:
send_image(port, x_test_normalized[image_no])

In [21]:
read_results(port)

(array([], dtype=float64), {})

In [64]:
read_raw_results(port)

		Layer_0_CONV_2D 		13001
		Layer_1_CONV_2D 		14946
		Layer_2_CONV_2D 		14947
		Layer_3_CONV_2D 		14946
		Layer_4_RESHAPE 		4511
		Layer_5_FULLY_CONNECTED 		3323
		Layer_6_SOFTMAX 		22
_start_report_ 
	Class 0 (0.1000000015) 	0x1.99999ap-4
	Class 1 (0.1000000015) 	0x1.99999ap-4
	Class 2 (0.1000000015) 	0x1.99999ap-4
	Class 3 (0.1000000015) 	0x1.99999ap-4
	Class 4 (0.1000000015) 	0x1.99999ap-4
	Class 5 (0.1000000015) 	0x1.99999ap-4
	Class 6 (0.1000000015) 	0x1.99999ap-4
	Class 7 (0.1000000015) 	0x1.99999ap-4
	Class 8 (0.1000000015) 	0x1.99999ap-4
	Class 9 (0.1000000015) 	0x1.99999ap-4
_end_report_ 


1

### Run a test verification of 50 images

In [14]:
mcu_predictions, benchmark_results = verify_accuracy_on_mcu(port, test=True)

print(mcu_predictions, benchmark_results)

[[0.         0.         0.         0.99609375 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.99609375 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.99609375 0.        ]
 [0.8203125  0.         0.         0.         0.         0.
  0.         0.         0.0234375  0.15625   ]
 [0.         0.         0.         0.         0.         0.
  0.99609375 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.99609375 0.         0.         0.        ]
 [0.         0.99609375 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.99609375 0.         0.         0.        ]
 [0.         0.         0.         0.99609375 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.99609375 0.         0.

### Run the verification of the whole test set

In [ ]:
mcu_predictions, benchmark_results = verify_accuracy_on_mcu(port)

#### Compare accuracies

In [16]:
tfl_model_selection = widgets.Dropdown(
    options=glob.glob(f"TFLite-model/*{model_name}*.tflite"),
    description='Select model:',
)
display(tfl_model_selection)

Dropdown(description='Select model:', options=('TFLite-model/01b_ResNet20_CIFAR-10_Q-full.tflite', 'TFLite-mod…

In [20]:
# TF
tf_model_predictions = tf_model.predict(x=x_test_normalized)
tf_model_accuracy = calc_accuracy(tf_model_predictions, y_test)

# TFL
tfl_model_predictions = tfl_predict(tfl_model_selection.value, x=x_test_normalized)
tfl_model_accuracy = calc_accuracy(tfl_model_predictions, y_test)

# TFLu
accurate_count = 0

for index in range(len(mcu_predictions)):
    if np.argmax(mcu_predictions[index]) == np.argmax(y_test[index]):
        accurate_count += 1

mcu_accuracy = accurate_count * 1.0 / len(mcu_predictions)

print("TF Lite Model on MCU Accuracy:\t", mcu_accuracy)
print("TF Lite Model Accuracy:\t\t", tfl_model_accuracy)
print('Original model accuracy:\t', tf_model_accuracy)
print()
loss_fn = tf.keras.losses.CategoricalCrossentropy(reduction='sum_over_batch_size')
mcu_crossentropy_loss = loss_fn(y_test[:10], mcu_predictions).numpy()

print("TF Lite Model on MCU cross entropy loss:\t", mcu_crossentropy_loss)
print("TF Lite Model cross entropy loss:\t\t", tfl_crossentropy_loss)
print('Original model cross entropy loss:\t\t', tf_model_loss)

TF Lite Model on MCU Accuracy:	 0.94
TF Lite Model Accuracy:		 0.9185
Original model accuracy:	 0.9185



ValueError: Shapes (10, 10) and (50, 10) are incompatible

In [68]:
calc_accuracy(tfl_model_predictions, y_test)

0.9179